Input

In [109]:
file_path = "input.txt"  # Path to the input file

with open(file_path, "r") as f:
    data = f.read()

from collections import OrderedDict

# dest source range
data = data.split('\n\n')
seeds = list(map(int, data[0].split(':')[1].strip().split()))

maps = []
for mp in data[1:]:
    mp = mp.split('\n')[1:]

    prev_map = {
        kp[1]: [kp[0], kp[2]]
        for line in mp
        if line.strip() and (kp := list(map(int, line.split())))
    }

    prev_map = OrderedDict(sorted(prev_map.items()))
    maps.append(prev_map)

Part 1

In [111]:
from bisect import bisect_right

def find_highest_key(t, mp):
    kys = list(mp.keys())
    idx = bisect_right(kys, t) - 1
    if idx >= 0:
        return kys[idx]
    return -1

curr_lvl = seeds

for mp in maps:
    nxt_lvl = []
    for k in curr_lvl:
        hk = find_highest_key(k, mp)
        if hk == -1 or (k >= hk + mp[hk][1]): rk = k
        else: rk = mp[hk][0] + (k - hk)
        nxt_lvl.append(rk)
    curr_lvl = nxt_lvl

sorted(curr_lvl)[0]

331445006

Part 2

In [113]:
# whoopingly large inputs, well F. Range mapping SMH

def find_highest_key(t, mp):
    kys = list(mp.keys())
    idx = bisect_right(kys, t) - 1
    if idx >= 0:
        return idx
    return -1  # No valid key found

curr_ranges = [[seeds[i], seeds[i] + seeds[i+1] - 1] for i in range(0, len(seeds), 2)]

for mp in maps:
    mpks = list(mp.keys())

    next_ranges = []
    for r in curr_ranges:
        new_ranges = []
        hkl = find_highest_key(r[0], mp)
        hkr = find_highest_key(r[1], mp)
        if hkl == -1:
            if hkr == -1: 
                new_ranges.append(r)
                next_ranges.extend(new_ranges)
                continue
            split_ks = mpks[hkl+1:hkr+1]
        else: split_ks = mpks[hkl:hkr+1]
        
        cnt = 0
        for k in split_ks:
            r_st = r[0] + cnt
            if r_st < k: 
                new_ranges.append([r_st, k-1])
                cnt += k - r_st
            st = mp[k][0] + (r[0] + cnt - k)
            en = mp[k][0] + min(mp[k][1] - 1, r[1] - k)

            if en >= st:
                new_ranges.append([st, en])
                cnt += en - st + 1
        
        if cnt < r[1] - r[0] + 1: 
            new_ranges.append([r[0] + cnt, r[1]])
        next_ranges.extend(new_ranges)
    
    curr_ranges = next_ranges

curr_ranges.sort()
curr_ranges[0][0]

6472060